In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

import jupyterlab_dash
import dash
import dash_html_components as html

from pathlib import Path

import plotly.express as px

import plotly.graph_objs as go

from dash.dependencies import Input, Output

import pandas as pd

In [ ]:
base_dir = Path().cwd().parent
processed_dir = base_dir/'data'/'processed'/'daily_report'

days = pd.date_range('03/01/2020', '04/02/2020', normalize=True)
days = days.strftime('%m-%d-%Y')

df = pd.DataFrame()
for day in days:
    temp = pd.read_csv(processed_dir/f'{day}.csv')
    temp.loc[:, 'date'] = pd.to_datetime(day)
    df = df.append(temp)

## generate dataframe

In [ ]:
def generate_dataframe():
    
    df = pd.DataFrame()  

    for day in days:
        temp = pd.read_csv(processed_dir/f'{day}.csv')
        temp.loc[:, 'date'] = pd.to_datetime(day)
        df = df.append(temp)
        
    return df

In [ ]:
df = generate_dataframe()

## generate date picker

In [ ]:
def generate_date_picker():   
    return dcc.DatePickerSingle(
        id='date-picker',
        min_date_allowed=dt(2020,3,1),
        max_date_allowed=dt(2020,4,2),
        initial_visible_month=dt(2020,3,15),
        date=str(dt(2020, 3, 15))
    )

## generate map

In [ ]:
def generate_map(df, date=None):
    
    if date is not None:
        df_map = df[df['date'] == pd.to_datetime(date)]     
    else:
        df_map = df[df['date'] == pd.to_datetime('03-15-2020')]
    
    choro = go.Choropleth(
        locations=df_map['country'],
        locationmode='country names',
        z=df_map['confirmed'],
        text=df_map['country'],
        autocolorscale=False,
        colorscale="YlOrRd",
        showscale=False,
    )
    
    fig = go.Figure(choro)
    
    fig.update_layout(
        # title_text = 'World Map',
        geo_scope='world', # limite map scope to USA
        margin={"r":10,"t":10,"l":10,"b":10},
        
    )
    
    fig.update_geos(
        resolution=110,
        showcoastlines=True, coastlinecolor="black",
        showland=False, landcolor="LightGreen",
        showocean=True, oceancolor="LightGrey",
        showlakes=False, lakecolor="Blue",
        showrivers=False, rivercolor="Blue"
)
    return fig

## generate confirmed case development

In [ ]:
def generate_ccd(df):
    pass

## generate table

In [ ]:
def generate_table(df,date=None, max_rows=5):
    
    if date is not None:
        df = df[df['date'] == pd.to_datetime(date)]     
    else:
        df = df[df['date'] == pd.to_datetime('03-15-2020')]
    
    columns = ['country', 'confirmed', 'deaths', 'recovered']
    
    df = df[columns].copy()
    df = df.sort_values(by=columns[1], ascending=False)
    
    table = go.Table(
        header={
            'values': [c.capitalize() for c in df[columns].columns],
            'fill_color': '#faf0e6',
            'align': 'left'
        },
        cells={
            'values': [df[c] for c in df[columns].columns],
            'fill_color': '#fdf5e6',
            'align': 'left',
        },
    )
    
    return go.Figure(table)

In [ ]:
from datetime import datetime as dt
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import re

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    
    # set page header
    html.H4(children='COVID-Dashboard'),
    
    # set date picker via calendar    
    html.Div(children=[
        generate_date_picker(),
    ],
        # style={'width': '49%', 'display': 'inline-block'}
    ),
    
    html.Div(children=[
        
        # set map graph 
        html.Div(
            dcc.Graph(id='graph-map', figure=generate_map(df)), 
            style={'width': '67%', 'display': 'inline-block'}
        ),
        
        # set country info table
        html.Div(
            dcc.Graph(id='table-info', figure=generate_table(df)),
            style={'width': '32%', 
                   'display': 'inline-block',
                   'overflowY': 'scroll'}
        )
    ],
    ),
    
    html.Div(children=[
        
       # set country info table
        html.Div(
            dcc.Graph(id='table-info2', figure=generate_table(df)), 
            style={'width': '33%', 'display': 'inline-block'}
        ),
        
        # set country info table
        html.Div(
            dcc.Graph(id='table-info3', figure=generate_table(df)), 
            style={'width': '33%', 'display': 'inline-block'}
        ),
        
        # set country info table
        html.Div(
            dcc.Graph(id='table-info4', figure=generate_table(df)), 
            style={'width': '33%', 'display': 'inline-block'}
        ),
    ],
    )
    
])

@app.callback(
    [Output('graph-map', 'figure'),
     Output('table-info', 'figure')],
    [Input('date-picker', 'date')])
def update_output(date):

    map_graph = generate_map(df, date=pd.to_datetime(date))
    table_info = generate_table(df, date=pd.to_datetime(date))
    
    return map_graph, table_info


viewer = jupyterlab_dash.AppViewer()
viewer.show(app)